In [55]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import math
import numpy as np
import scipy.stats
import copy
import random
import operator

In [57]:
class Chromossome:
    def __init__(self, algorithm, **hyperparameter_range):
        self.hyperparameter_range = hyperparameter_range
        self.classifier = algorithm()
        self.mutate()
        self.fitness = 0
        
    def fit(self, X, y):
        self.classifier.fit(X, y)
        
    def predict(self, X):
        return self.classifier.predict(X)
    
    def mutate(self, n_positions=None):
        param = {}        
        if not n_positions or n_positions>len(self.hyperparameter_range):
            n_positions = len(self.hyperparameter_range)
        mutation_positions = random.sample(range(0, len(self.hyperparameter_range)), n_positions)
        i = 0
        for hyperparameter, h_range in self.hyperparameter_range.items():
            if i in mutation_positions:
                if isinstance(h_range[0], str):
                    param[hyperparameter] = random.choice(h_range)
                elif isinstance(h_range[0], float):
                    param[hyperparameter] = random.uniform(h_range[0], h_range[1]+1)
                else:
                    param[hyperparameter] = random.randint(h_range[0], h_range[1]+1)
            i+= 1
        
        self.classifier.set_params(**param)

In [58]:
class DiversityEnsembleClassifier:
    def __init__(self, algorithms, population_size = 100, max_epochs = 100, random_state=None):
        self.population_size = population_size
        self.max_epochs = max_epochs
        self.population = []
        self.random_state = random_state
        random.seed(random_state)
        for algorithm, hyperparameters in algorithms.items():
            for i in range(0, math.ceil(population_size/len(algorithms.keys()))):
                self.population.append(Chromossome(algorithm, **hyperparameters))
        
    def fit(self, X, y):
        print('Running epoch ', end='')
                
        kf = KFold(n_splits=5, random_state=self.random_state)
        
        for epoch in range(self.max_epochs):
            random.seed(self.random_state)
            print(epoch, end='...')
            predictions = np.empty([2*self.population_size, y.shape[0]])
            
            for i in range(0, self.population_size):
                new_chromossome = copy.deepcopy(self.population[i])
                new_chromossome.mutate(1)
                self.population.append(new_chromossome)
                
            for i in range(2*self.population_size):
                chromossome = self.population[i]
                for train, val in kf.split(X):
                    chromossome.fit(X[train], y[train])
                    predictions[i][val] = np.equal(chromossome.predict(X[val]), y[val])
                 
            distances = np.zeros(2*self.population_size)
            pop_fitness = predictions.sum(axis=1)
            target_chromossome = np.argmin(pop_fitness)
            new_population = []            
            
            diversity  = np.zeros(2*self.population_size)
            selected = [target_chromossome]
            for i in range(0, self.population_size-1):
                distances[target_chromossome] = float('-inf')
                d_i = np.logical_xor(predictions, predictions[target_chromossome]).sum(axis=1)
                distances += d_i
                diversity += d_i
                target_chromossome = np.argmax(distances)
                selected.append(target_chromossome)
                self.population[target_chromossome].fitness = pop_fitness[target_chromossome]
            
            for x in selected:
                new_population.append(self.population[x])     
                
            diversity = diversity[selected].sum()
            print(diversity)
            self.population =new_population
            
        for chromossome in self.population:
            chromossome.fit(X, y)
            
    def predict(self, X):
        predictions = np.empty((self.population_size, len(X)))
        y = np.empty(len(X))
        for chromossome in range(0, self.population_size):
            predictions[chromossome] = self.population[chromossome].predict(X)
        for i in range(0, len(X)):
            pred = {}
            for j in range(0, self.population_size):
                if predictions[j][i] in pred:
                    pred[predictions[j][i]] += self.population[j].fitness
                else: 
                    pred[predictions[j][i]]  = self.population[j].fitness
            y[i] = max(pred.items(), key=operator.itemgetter(1))[0]
        return y
        """
        predictions = np.empty([self.population_size, X.shape[0]])
        for i in range(0, self.population_size):
            predictions[i] = self.population[i].predict(X)
        return scipy.stats.mode(predictions, axis=0).mode[0]
        """

In [59]:
wine = datasets.load_wine()

In [60]:
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2, random_state=40)

In [66]:
alg = {
            KNeighborsClassifier: {'n_neighbors':[1, 107]},
            SVC: {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], 
                  'degree' : [0, 3]
                  },
            DecisionTreeClassifier: {'min_samples_leaf':[1, 100], 'max_depth':[3, 20]},
            RandomForestClassifier: {'min_samples_leaf':[1, 100], 'max_depth':[3, 20],
                                     'n_estimators':[100, 100]},
            GaussianNB: {},
            LinearDiscriminantAnalysis: {}
    
      }
dec = DiversityEnsembleClassifier(alg, population_size=100, max_epochs=19, random_state=42)
dec.fit(X_train,  y_train)
print(accuracy_score(y_test, dec.predict(X_test)))

Running epoch 0...

C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


479092.0
1...

C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Ren

490642.0
2...

C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Ren

497696.0
3...

C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)
C:\Users\AlandeAguiar\Anaconda2\envs\python3\lib\site-packages\sklearn\discriminant_analysis.py:442: UserWarning: The priors do not sum to 1. Renormalizing
  UserWarning)


502587.0
4...505215.0
5...507684.0
6...509456.0
7...511330.0
8...513364.0
9...514066.0
10...514066.0
11...514066.0
12...514066.0
13...514072.0
14...515159.0
15...515342.0
16...515369.0
17...515413.0
18...515436.0
0.8055555555555556


In [65]:
#Inspect the classifiers
print([x.classifier for x in dec.population])

[SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=4, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=31, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=56, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'), KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski'

In [38]:
#Check number of unique classifiers
unique = set([str(x.classifier.get_params()) for x in dec.population])
len(unique)

41

In [19]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)
print(accuracy_score(y_test, rf.predict(X_test)))

1.0


In [ ]:
np.empty()